# Imports

In [1]:
# If you need some install, uncomment the code bellow 

# !pip install torch==1.4
# !pip install torchvision
# !pip install ipdb

# !pip install torchaudio
# !pip install PyDrive
# !pip install soundfile

In [1]:
import torch
import torchaudio
from matplotlib import pyplot as plt
import numpy as np
from torch import Tensor
import os
from typing import Tuple

import ipdb

# from torchaudio.datasets import YESNO, LIBRISPEECH
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import pdb, traceback, sys


from torchaudio.datasets.utils import (
  download_url,
  extract_archive,
  walk_files,
)

# MaseseSpeech 2h

In [ ]:
# Train link Drive
# https://drive.google.com/file/d/1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR/view?usp=sharing
    
# wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR" -O train-clean.tar.xz && rm -rf /tmp/cookies.txt
    
# # Valid link Drive
# https://drive.google.com/file/d/1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP/view?usp=sharing
# https://drive.google.com/file/d/1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP/view?usp=sharing
    
# wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP" -O dev-clean.tar.xz && rm -rf /tmp/cookies.txt

In [ ]:
# !ls

In [4]:
# %cd ..

In [5]:
!wget  https://github.com/Kabongosalomon/MaseseSpeech/raw/master/voalingala.tar.xz -O voalingala.tar.xz

extract_archive("voalingala.tar.xz")
!rm -r "voalingala.tar.xz"

--2020-07-04 17:46:18--  https://github.com/Kabongosalomon/MaseseSpeech/raw/master/voalingala.tar.xz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Kabongosalomon/MaseseSpeech/master/voalingala.tar.xz [following]
--2020-07-04 17:46:18--  https://raw.githubusercontent.com/Kabongosalomon/MaseseSpeech/master/voalingala.tar.xz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32168212 (31M) [application/octet-stream]
Saving to: ‘voalingala.tar.xz’

voalingala.tar.xz   100%[===================>]  30.68M  64.3MB/s    in 0.5s    

2020-07-04 17:46:20 (64.3 MB/s) - ‘voalingala.tar.xz’ saved [32168212/32168212

In [6]:
# !mkdir MaseseSpeech
# !mkdir MaseseSpeech

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CfqHBiOEVnJiuwZoqBJMlGO-N97JD3sR" -O MaseseSpeech/train-clean.tar.xz && rm -rf /tmp/cookies.txt
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Y1CiB7TbrGdghVokwMTBQA1fRza2NZUP" -O MaseseSpeech/dev-clean.tar.xz && rm -rf /tmp/cookies.txt

# extract_archive("MaseseSpeech/train-clean.tar.xz")
# extract_archive("MaseseSpeech/dev-clean.tar.xz")

# !rm -r "MaseseSpeech/train-clean.tar.xz"
# !rm -r "MaseseSpeech/dev-clean.tar.xz"

In [7]:
URL = "train-clean"
FOLDER_IN_ARCHIVE = "MaseseSpeech"
# BASE_URL = "https://dl.fbaipublicfiles.com/librispeech_100h_mp3/"
# _CHECKSUMS = {
#   BASE_URL + "dev-clean.tar.gz":
#   "076916a8f9c61951c5d2e6efaa8d2188232fcf860eec8c074e46edf4fac9623e",
#   BASE_URL + "test-clean.tar.gz":
#   "3c171e2f1e377e4993c2dbe6bff3f01cd324c0ed462f4de6c78737402a7dbedd",
#   BASE_URL + "train-clean-100.tar.gz":
#   "7bfbefc680d25ba3a82798ce32c287ea0e82932af1b1f864fae71fb52d2f41f0",
# }


def load_masesespeech_item(fileid: str, 
                          path: str, 
                          ext_audio: str, 
                          ext_txt: str) -> Tuple[Tensor, int, str, int, int, int]:
    
    book_id, chapter_id, utterance_id = fileid.split("-")
    
    file_text = book_id + "-" + chapter_id + ext_txt
    file_text = os.path.join(path, book_id, chapter_id, file_text)
    
    fileid_audio = book_id + "-" + chapter_id + "-" + utterance_id
    file_audio = fileid_audio + ext_audio
    file_audio = os.path.join(path, book_id, chapter_id, file_audio)
    
#     ipdb.set_trace()
    try :
        # Load audio
        waveform, sample_rate = torchaudio.load(file_audio)

        # Load text
        with open(file_text) as ft:
            for line in ft:
                fileid_text, utterance = line.strip().split(" ", 1) # this takes the first space split
                if fileid_audio == fileid_text:
                    # stop when we found the text corresponding to 
                    # the audio ID
                    break
            else:
              # Translation not found
              raise FileNotFoundError("Translation not found for " + fileid_audio)
    except:
        print(file_audio) # this is for debugging purpose 
        print(waveform)   # to show which file may have an issue 
        pass
#         traceback.print_exc()
                
    return (
        waveform,
        sample_rate,
        utterance,
        int(book_id),
        int(chapter_id),
        int(utterance_id)
        )


class MASESESPEECH_2H_MP3(Dataset):
    """
    Create a Dataset for MaseseSpeech. Each item is a tuple of the form:
    waveform, utterance, chapter_id, verse_id, utterance_id
    """
    
    _ext_txt = ".trans.txt"
    _ext_audio = ".wav"
    
    def __init__(self,
                 root: str,
                 mode: str = "MaseseSpeech/train-clean",
                 folder_in_archive: str = FOLDER_IN_ARCHIVE,
#                  download: bool = False
                ) -> None:
#         if url in [
#             "dev-clean",
# #             "test-clean",
#             "train-clean",]:
            
#             ext_archive = ".tar.xz"
#             base_url = BASE_URL
#             url = os.path.join(base_url, url + ext_archive)

#         basename = os.path.basename(url)
#         archive = os.path.join(root, basename)

#         basename = basename.split(".")[0]
#         folder_in_archive = os.path.join(folder_in_archive, basename)
        
        
        self._path = mode

#         if download:
#             if not os.path.isdir(self._path):
#                 if not os.path.isfile(archive):
#                     checksum = _CHECKSUMS.get(url, None)
#                     download_url(url, root, hash_value=checksum)
        
        walker = walk_files(
          self._path, suffix=self._ext_audio, prefix=False, remove_suffix=True
        )
        self._walker = list(walker)
    def __getitem__(self, n: int) -> Tuple[Tensor, int, str, int, int, int]:
        fileid = self._walker[n]
        return load_masesespeech_item(fileid, self._path, self._ext_audio, self._ext_txt)
    
    def __len__(self) -> int:
        return len(self._walker)

In [8]:
masese_train = MASESESPEECH_2H_MP3(".", mode = "MaseseSpeech/train-clean")
masese_dev = MASESESPEECH_2H_MP3(".", mode = "MaseseSpeech/dev-clean")

In [9]:
# just so you get an idea of the format 
print(next(iter(masese_train)))
print(next(iter(masese_dev)))

(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0218, 0.0149, 0.0132]]), 16000, 'BILOKO YA MOTUYA OYO EUTI NA MAKAMBO YA MPAMBA EKÓMAKA MOKE,  KASI MOTO OYO AYANGANISAKA NA LOBƆKƆ NDE AKOLISAKA YANGO.', 20, 13, 10)
(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0248, 0.0237, 0.0286]]), 16000, 'MPO BOLAI YA MIKOLO, BAMBULA YA BOMOI  NÁ KIMYA EKOBAKISAMELA YO.', 20, 3, 1)


In [10]:
# Use this is your acoustic model is outputting letters
special_caracters = "Ɛ,;.ƆÁÓ-?:*É—“”!Í(10)"    # Special caracters special to Lingala and this dataset
tokens_list = list(" ABCDEFGHIJKLMNOPQRSTUVWXYZ"+special_caracters)
tokens_set = set(tokens_list)
print(tokens_list)

[' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Ɛ', ',', ';', '.', 'Ɔ', 'Á', 'Ó', '-', '?', ':', '*', 'É', '—', '“', '”', '!', 'Í', '(', '1', '0', ')']


In [11]:
def collate_fn_libri(batch):
    #print(batch)
    tensors = [b[0].t() for b in batch if b]
    tensors_len = [len(t) for t in tensors]
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors.transpose(1, -1)
    
    transcriptions = [list(b[2].replace("'", " ")) for b in batch if b]
    targets = [torch.tensor([tokens_list.index(e) for e in t]) for t in transcriptions]
    targets_len = [len(t) for t in targets]
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
    
    return tensors, targets, torch.tensor(tensors_len), torch.tensor(targets_len)

# train_set = torch.utils.data.DataLoader(masese_train, batch_size=900, shuffle=True,
#                                         num_workers=4, collate_fn=collate_fn_libri)

In [12]:
# torchaudio.load("MaseseSpeech/train-clean/020/008/020-008-017.wav")

In [13]:
# print(next(iter(train_set)))
train_set = torch.utils.data.DataLoader(masese_train, batch_size=64, shuffle=True,
                                        num_workers=2, collate_fn=collate_fn_libri)

In [14]:
def collate_fn_libri(batch):
    #print(batch)
    tensors = [b[0].t() for b in batch if b]
    tensors_len = [len(t) for t in tensors]
    tensors = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
    tensors = tensors.transpose(1, -1)
    
    transcriptions = [list(b[2].replace("'", " ")) for b in batch if b]
    targets = [torch.tensor([tokens_list.index(e) for e in t]) for t in transcriptions]
    targets_len = [len(t) for t in targets]
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
    
    return tensors, targets, torch.tensor(tensors_len), torch.tensor(targets_len)

# valid_set = torch.utils.data.DataLoader(masese_dev, batch_size=1500, shuffle=True,
#                                         num_workers=4, collate_fn=collate_fn_libri)

In [15]:
# print(next(iter(valid_set)))

In [16]:
# ! git clone https://github.com/facebookresearch/CPC_audio.git  #

In [17]:
%cd CPC_audio/

/root/AMMI-Speech-Lig-Aikuma/Dataset/Project/CPC_audio


In [18]:
# !ls

In [19]:
# !%cd /CPC_audio
# !python setup.py develop

## Exercise 1 : Building the model

In this exercise, we will build and train a small CPC model using the repository CPC_audio.

In [20]:
# %cd /content/CPC_audio
from cpc.model import CPCEncoder, CPCAR
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

DIM_ENCODER= 256 #256
DIM_CONTEXT= 256 #256
KEEP_HIDDEN_VECTOR=False
N_LEVELS_CONTEXT=1
CONTEXT_RNN="LSTM"
N_PREDICTIONS=12
LEARNING_RATE=2e-4
N_NEGATIVE_SAMPLE =128

In [21]:
encoder = CPCEncoder(DIM_ENCODER)
context = CPCAR(DIM_ENCODER,
                DIM_CONTEXT,
                KEEP_HIDDEN_VECTOR,
                N_LEVELS_CONTEXT,
                mode="CONTEXT_RNN")

In [22]:
# Several functions that will be necessary to load the data later
from cpc.dataset import findAllSeqs, AudioBatchData, parseSeqLabels
SIZE_WINDOW = 20480
BATCH_SIZE=8
def load_dataset(path_dataset, file_extension='.wav', phone_label_dict=None):
    data_list, speakers = findAllSeqs(path_dataset, extension=file_extension)
    dataset = AudioBatchData(path_dataset, SIZE_WINDOW, data_list, phone_label_dict, len(speakers))
    return dataset

In [23]:
class CPCModel(torch.nn.Module):

    def __init__(self,
                 encoder,
                 AR):

        super(CPCModel, self).__init__()
        self.gEncoder = encoder
        self.gAR = AR

    def forward(self, batch_data):
        

        encoder_output = self.gEncoder(batch_data)
        #print(encoder_output.shape)
        # The output of the encoder data does not have the good format 
        # indeed it is Batch_size x Hidden_size x temp size
        # while the context requires Batch_size  x temp size x Hidden_size
        # thus you need to permute
        context_input = encoder_output.permute(0, 2, 1)

        context_output = self.gAR(context_input)
        #print(context_output.shape)
        return context_output, encoder_output

In [23]:
# !ls ..

In [24]:
audio = torchaudio.load(
#         "../voalingala/20200611/20200611-160000-VCD361-program_16k.mp3")[0]
        "../MaseseSpeech/train-clean/020/001/020-001-013.wav")[0]
        

audio = audio.view(1, 1, -1)

cpc_model = CPCModel(encoder, context).to(device)
context_output, encoder_output = cpc_model(audio.to(device))

In [25]:
audio

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.1118, 0.0728, 0.0479]]])

In [26]:
# from cpc.dataset import parseSeqLabels
# from cpc.feature_loader import loadModel

# checkpoint_path = 'checkpoint_data/checkpoint_30.pt'
# cpc_model, HIDDEN_CONTEXT_MODEL, HIDDEN_ENCODER_MODEL = loadModel([checkpoint_path])
# cpc_model = cpc_model.cuda()

## Exercise 2 : CPC loss

We will define a class ```CPCCriterion``` which will hold the prediction networks $\phi_k$ defined above and perform the classification loss $\mathcal{L}_c$.

a) In this exercise, the $\phi_k$ will be a linear transform, ie:

\\[ \phi_k(c_t) = \mathbf{A}_k c_t\\]

Using the class [torch.nn.Linear](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear), define the transformations $\phi_k$ in the code below and complete the function ```get_prediction_k``` which computes $\phi_k(c_t)$ for a given batch of vectors $c_t$.

b) Using both ```get_prediction_k```  and ```sample_negatives``` defined below, write the forward function which will take as input two batches of features $c_t$ and $g_t$ and outputs the classification loss $\mathcal{L}_c$ and the average acuracy for all predictions. 

In [27]:
# Exercice 2: write the CPC loss
# a) Write the negative sampling (with some help)
# ERRATUM: it's really hard, the sampling will be provided

class CPCCriterion(torch.nn.Module):
    
    def __init__(self,
                 K,
                 dim_context,
                 dim_encoder,
                 n_negative):
        
        super(CPCCriterion, self).__init__()
        
        self.K_ = K
        self.dim_context = dim_context
        self.dim_encoder = dim_encoder
        self.n_negative = n_negative

        self.predictors = torch.nn.ModuleList()
        
        for k in range(self.K_):
            # TO COMPLETE !
            # A affine transformation in pytorch is equivalent to a nn.Linear layer
            # To get a linear transformation you must set bias=False
            # input dimension of the layer = dimension of the encoder
            # output dimension of the layer = dimension of the context
            self.predictors.append(torch.nn.Linear(dim_context, dim_encoder, bias=False))
        
    def get_prediction_k(self, context_data):
        #TO COMPLETE !
        output = [] 
        # For each time step k
        for k in range(self.K_):
            # We need to compute phi_k = A_k * c_t
            phi_k = self.predictors[k](context_data)
            output.append(phi_k)

        return output
    
    def sample_negatives(self, encoded_data):
        r"""
        Sample some negative examples in the given encoded data.
        Input:
        - encoded_data size: B x T x H
        Returns
        - outputs of size B x (n_negative + 1) x (T - K_) x H
          outputs[:, 0, :, :] contains the positive example
          outputs[:, 1:, :, :] contains negative example sampled in the batch
        - labels, long tensor of size B x (T - K_)
          Since the positive example is always at coordinates 0 for all sequences 
          in the batch and all timestep in the sequence, labels is just a tensor
          full of zeros !
        """
        batch_size, time_size, dim_encoded = encoded_data.size()
        window_size = time_size - self.K_
        outputs = []

        neg_ext = encoded_data.contiguous().view(-1, dim_encoded)
        n_elem_sampled = self.n_negative * window_size * batch_size
        # Draw nNegativeExt * batchSize negative samples anywhere in the batch
        batch_idx = torch.randint(low=0, high=batch_size,
                                  size=(n_elem_sampled, ),
                                  device=encoded_data.device)

        seq_idx = torch.randint(low=1, high=time_size,
                                size=(n_elem_sampled, ),
                                device=encoded_data.device)

        base_idx = torch.arange(0, window_size, device=encoded_data.device)
        base_idx = base_idx.view(1, 1, window_size)
        base_idx = base_idx.expand(1, self.n_negative, window_size)
        base_idx = base_idx.expand(batch_size, self.n_negative, window_size)
        seq_idx += base_idx.contiguous().view(-1)
        seq_idx = torch.remainder(seq_idx, time_size)

        ext_idx = seq_idx + batch_idx * time_size
        neg_ext = neg_ext[ext_idx].view(batch_size, self.n_negative,
                                        window_size, dim_encoded)
        label_loss = torch.zeros((batch_size, window_size),
                                  dtype=torch.long,
                                  device=encoded_data.device)

        for k in range(1, self.K_ + 1):
            # Positive samples
            if k < self.K_:
                pos_seq = encoded_data[:, k:-(self.K_-k)]
            else:
                pos_seq = encoded_data[:, k:]
            pos_seq = pos_seq.view(batch_size, 1, pos_seq.size(1), dim_encoded)
            full_seq = torch.cat((pos_seq, neg_ext), dim=1)
            outputs.append(full_seq)

        return outputs, label_loss
    
    def forward(self, encoded_data, context_data):

        # TO COMPLETE:
        # Perform the full cpc criterion
        # Returns 2 values:
        # - the average classification loss avg_loss
        # - the average classification acuracy avg_acc

        # Reminder : The permuation !
        encoded_data = encoded_data.permute(0, 2, 1)

        # First we need to sample the negative examples
        negative_samples, labels = self.sample_negatives(encoded_data)

        # Then we must compute phi_k
        phi_k = self.get_prediction_k(context_data)

        # Finally we must get the dot product between phi_k and negative_samples 
        # for each k

        #The total loss is the average of all losses
        avg_loss = 0

        # Average acuracy
        avg_acc = 0

        for k in range(self.K_):
            B, N_sampled, S_small, H = negative_samples[k].size() 
            B, S, H = phi_k[k].size()
            
            # As told before S = S_small + K. For segments too far in the sequence
            # there are no positive exmples anyway, so we must shorten phi_k
            phi = phi_k[k][:, :S_small]
            
            # Now the dot product
            # You have several ways to do that, let's do the simple but non optimal 
            # one
            # pytorch has a matrix product function https://pytorch.org/docs/stable/torch.html#torch.bmm
            # But it takes only 3D tensors of the same batch size !
            # To begin negative_samples is a 4D tensor ! 
            # We want to compute the dot product for each features, of each sequence
            # of the batch. Thus we are trying to compute a dot product for all
            # B* N_sampled * S_small 1D vector of negative_samples[k]
            # Or, a 1D tensor of size H is also a matrix of size 1 x H
            # Then, we must view it as a 3D tensor of size (B* N_sampled * S_small, 1, H)
            negative_sample_k  =  negative_samples[k].view(B* N_sampled* S_small, 1, H)
            
            # But now phi and negative_sample_k no longer have the same batch size !
            # No worries, we can expand phi so that each sequence of the batch
            # is repeated N_sampled times
            phi = phi.view(B, 1,S_small, H).expand(B, N_sampled, S_small, H)
            
            # And now we can view it as a 3D tensor 
            phi  = phi.contiguous().view(B * N_sampled * S_small, H, 1)
            
            # We can finally get the dot product !
            scores = torch.bmm(negative_sample_k, phi)
            
            # Dot_product has a size (B * N_sampled * S_small , 1, 1)
            # Let's reorder it a bit
            scores = scores.reshape(B, N_sampled, S_small)
            
            # For each elements of the sequence, and each elements sampled, it gives 
            # a floating score stating the likelihood of this element being the 
            # true one.
            # Now the classification loss, we need to use the Cross Entropy loss
            # https://pytorch.org/docs/master/generated/torch.nn.CrossEntropyLoss.html
            
            # For each time-step of each sequence of the batch 
            # we have N_sampled possible predictions. 
            # Looking at the documentation of torch.nn.CrossEntropyLoss
            # we can see that this loss expect a tensor of size M x C where 
            # - M is the number of elements with a classification score
            # - C is the number of possible classes
            # There are N_sampled candidates for each predictions so
            # C = N_sampled 
            # Each timestep of each sequence of the batch has a prediction so
            # M = B * S_small
            # Thus we need an input vector of size B * S_small, N_sampled
            # To begin, we need to permute the axis
            scores = scores.permute(0, 2, 1) # Now it has size B , S_small, N_sampled
            
            # Then we can cast it into a 2D tensor
            scores = scores.reshape(B * S_small, N_sampled)
            
            # Same thing for the labels 
            labels = labels.reshape(B * S_small)
            
            # Finally we can get the classification loss
            loss_criterion = torch.nn.CrossEntropyLoss()
            loss_k = loss_criterion(scores, labels)
            avg_loss+= loss_k
            
            # And for the acuracy
            # The prediction for each elements is the sample with the highest score
            # Thus the tensors of all predictions is the tensors of the index of the 
            # maximal score for each time-step of each sequence of the batch
            predictions = torch.argmax(scores, 1)
            acc_k  = (labels == predictions).sum() / (B * S_small)
            avg_acc += acc_k

        # Normalization
        avg_loss = avg_loss / self.K_
        avg_acc = avg_acc / self.K_
        
        return avg_loss , avg_acc

In [28]:
audio = torchaudio.load(
    "../MaseseSpeech/train-clean/020/001/020-001-013.wav")[0]
#     "../voalingala/20200611/20200611-160000-VCD361-program_16k.mp3")[0]

audio = audio.view(1, 1, -1)
cpc_criterion = CPCCriterion(N_PREDICTIONS, DIM_CONTEXT, 
                             DIM_ENCODER, N_NEGATIVE_SAMPLE).to(device)
context_output, encoder_output = cpc_model(audio.to(device))
loss, avg = cpc_criterion(encoder_output,context_output)

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [29]:
loss

tensor(5.3424, device='cuda:0', grad_fn=<DivBackward0>)

# Exercise 3: Full training loop !

You have the model, you have the criterion. All you need now are a data loader and an optimizer to run your training loop.

We will use an Adam optimizer:

In [30]:
parameters = list(cpc_criterion.parameters()) + list(cpc_model.parameters())
optimizer = torch.optim.Adam(parameters, lr=LEARNING_RATE)

In [ ]:
# dataset_train = load_dataset('../voalingala/train',
#                              file_extension='.mp3')
# dataset_val = load_dataset('../voalingala/val',
#                              file_extension='.mp3')

dataset_train = load_dataset('../MaseseSpeech/train-clean')
dataset_val = load_dataset('../MaseseSpeech/dev-clean')

data_loader_train = dataset_train.getDataLoader(BATCH_SIZE, "speaker", True)
data_loader_val = dataset_train.getDataLoader(BATCH_SIZE, "sequence", False)

15it [00:00, 5955.56it/s]

Saved cache file at ../MaseseSpeech/train-clean/_seqs_cache.txt



363it [00:00, 855355.25it/s]

Checking length...
Done, elapsed: 0.029 seconds
Scanned 363 sequences in 0.03 seconds
1 chunks computed
Joining pool
